# Wandb를 활용한 실험관리
이번 예제에서는 pytorch를 이용해 cifar10 분류문제를 푸는 과정을 기록해보겠습니다
- 참고자료: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py

## 1. 데이터 및 모델 설정

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
!pip install -U ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
#!!!이부분에서 Floatprogress not found 에러가 뜨면, 노트북을 재시작해주세요!!!!

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x




## 2. Wandb 설정

In [ ]:
!pip install wandb

!wandb login

### Wandb에 등록할 프로젝트 및 현재 훈련(Run) 관련 정보기록

In [ ]:
import wandb

config = dict (
  learning_rate = 0.01,
  momentum = 0.2,
  architecture = "CNN",
  dataset_id = "CIFAR10",
)

run = wandb.init(
  project="cifar",
  name = "my_test_01", #다른 테스트를 진행해보고 싶으면 이 이름 부분을 바꿔서 기록해주세요!
  notes="base line test",
  tags=["baseline", "cifar"],
  config=config,
)

## 위에서 정의한 모델과 wandb 세팅을 활용한 기록
- 기본적으로 wandb.log 함수로 기록을 진행합니다.

In [ ]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = Net()
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=config['learning_rate'], momentum=config['momentum'])
for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 100))
            
            ##wandb 로깅
            wandb.log({"training_batch_loss":running_loss / 100})
            running_loss = 0.0

    #VALIDATION
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    
    ## wandb 로깅
    ## 아래와 같이 다양한 정보들을 로깅 할 수 있습니다.
    wandb.log({"valid_acc": correct / total, "epoch": epoch,
           "inputs": wandb.Image(images), #마지막 배치의 이미지
           "logits": wandb.Histogram(outputs.to('cpu')), #마지막 배치의 아웃풋
           })

print('Finished Training')

## 훈련 결과 출력

In [ ]:

run.finish()


<img src="image-asset/screenshot_001.png"/>
위와 같은 그림을 wandb 웹사이트에서 확인하시면 성공!

## Wandb Sweep 
Wandb Sweep 기능을 활용하여, 목표 metric과 하이퍼파라미터간의 관계 확인하기
- 참조: https://docs.wandb.ai/guides/sweeps

In [ ]:
import wandb

sweep_config = {
    "name" : "my-sweep",
    "method" : "random", #grid, random
    "metric": { #목표로 삶을 매트릭에 대한 정보를 입력합니다.
        "name": "valid_acc",
        "goal": "maximize"
    },
    "parameters" : { #실험해볼 하이퍼 파라미터의 조합입니다.
        "epochs" : {
            "values" : [2, 3, 4]
        },
        "learning_rate" :{
            "values" : [0.001, 0.002, 0.003, 0.004]
        },
        "momentum" :{
            "min": 0.5,
            "max": 0.9
        }
      }
    }

sweep_id = wandb.sweep(sweep_config, project="cifar-hyperparameter-tunning")

In [ ]:
def train():
    with wandb.init() as run:
        config = wandb.config
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        net = Net()
        net.to(device)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=config['learning_rate'], momentum=config['momentum'])        
        for epoch in range(config["epochs"]):

            running_loss = 0.0
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                optimizer.zero_grad()

                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if i % 100 == 99:    # print every 10 mini-batches
                    print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 100))

                    ##wandb 로깅
                    wandb.log({"training_batch_loss":running_loss / 100})
                    running_loss = 0.0

            #VALIDATION
            correct = 0
            total = 0
            with torch.no_grad():
                for data in testloader:
                    images, labels = data[0].to(device), data[1].to(device)

                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

            ## wandb 로깅
            ## 아래와 같이 다양한 정보들을 로깅 할 수 있습니다.
            wandb.log({"valid_acc": correct / total, "epoch": epoch,
                   "inputs": wandb.Image(images), #마지막 배치의 이미지
                   "logits": wandb.Histogram(outputs.to('cpu')), #마지막 배치의 아웃풋
                   })

            print('Finished Training')

count = 5 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

Sweep 페이지에서 아래와 같은 정보들을 확인하시면 성공!
<img src="image-asset/screenshot_002.png"/>